# In-class Demo: Importing Data

This notebook demonstrates reading a variety of data sources into Python using Pandas.


In [2]:
from pathlib import Path
import pandas as pd, json
BASE = Path().resolve().parents[0] if (Path().resolve().name == "notebooks") else Path().resolve()
DATA = BASE / "data" / "demo"
print("DATA =", DATA)

DATA = /Users/besee/Library/CloudStorage/OneDrive-Personal/Documents/STAT 386/importing-data-class/data/demo


In [3]:
# --- CSV
csv_path = DATA / "csv" / "students.csv"
df_csv = pd.read_csv(csv_path)
print(df_csv.head())
# Common args: sep, header, index_col, nrows, encoding, usecols, dtype, compression
df_subset = pd.read_csv(csv_path, usecols=["id","name"], dtype={"id":"int64"})
print(df_subset.dtypes)

   id               name major   gpa  grad_year
0   1       Ada Lovelace    CS  3.90       1843
1   2        Alan Turing  Math  3.80       1934
2   3       Grace Hopper  EECS  3.95       1930
3   4    Edsger Dijkstra  Math  3.70       1956
4   5  Katherine Johnson  Math  3.92       1937
id       int64
name    object
dtype: object


In [4]:
# --- TSV (tab-delimited)
tsv_path = DATA / "tsv" / "sales.tsv"
df_tsv = pd.read_csv(tsv_path, sep="\t")
print(df_tsv.head())

  order_id        date product  quantity  price
0     A001  2024-01-05  Widget         3  19.99
1     A002  2024-01-06  Gadget         1  29.50
2     A003  2024-01-06  Widget         2  19.99
3     A004  2024-01-07  Doodad         5   3.75
4     A005  2024-01-08  Widget         1  19.99


In [5]:
# --- Compressed CSV (gzip)
gz_path = DATA / "csv_gz" / "students.csv.gz"
df_gz = pd.read_csv(gz_path, compression="gzip")
print(df_gz.head())

   id               name major   gpa  grad_year
0   1       Ada Lovelace    CS  3.90       1843
1   2        Alan Turing  Math  3.80       1934
2   3       Grace Hopper  EECS  3.95       1930
3   4    Edsger Dijkstra  Math  3.70       1956
4   5  Katherine Johnson  Math  3.92       1937


In [16]:
# --- Excel (may require 'openpyxl')
from pip import openpyxl
excel_path = DATA / "excel" / "grades.xlsx"
try:
    df_excel = pd.read_excel(excel_path)  # default: first sheet
    print(df_excel.head())
    all_sheets = pd.read_excel(excel_path, sheet_name=None)
    print("Sheets:", list(all_sheets))
except Exception as e:
    print("Excel read failed:", e)

ImportError: cannot import name 'openpyxl' from 'pip' (/Users/besee/.virtualenvs/r-tensorflow/lib/python3.11/site-packages/pip/__init__.py)

In [15]:
import json
from pandas import json_normalize

# --- JSON (flat)
people_path = DATA / "json" / "people.json"
with open(people_path, "r", encoding="utf-8") as f:
    people = json.load(f)
print(people)

# --- JSON (nested) with json_normalize
nested_path = DATA / "json" / "orders_nested.json"
with open(nested_path, "r", encoding="utf-8") as f:
    orders = json.load(f)
print(orders)

flat = json_normalize(orders, record_path="items", meta=["order_id", ["customer","id"], ["customer","name"]])
print(flat.head())

[{'name': 'John', 'age': 30, 'city': 'New York'}, {'name': 'Jane', 'age': 27, 'city': 'Chicago'}, {'name': 'Carlos', 'age': 35, 'city': 'Austin'}]
[{'order_id': 'A100', 'customer': {'id': 1, 'name': 'Alice'}, 'items': [{'sku': 'WID-001', 'qty': 2, 'price': 9.99}, {'sku': 'GAD-010', 'qty': 1, 'price': 29.5}], 'shipping': {'city': 'Boston', 'state': 'MA'}}, {'order_id': 'A101', 'customer': {'id': 2, 'name': 'Bob'}, 'items': [{'sku': 'WID-001', 'qty': 1, 'price': 9.99}, {'sku': 'DD-404', 'qty': 5, 'price': 3.75}], 'shipping': {'city': 'Seattle', 'state': 'WA'}}]
       sku  qty  price order_id customer.id customer.name
0  WID-001    2   9.99     A100           1         Alice
1  GAD-010    1  29.50     A100           1         Alice
2  WID-001    1   9.99     A101           2           Bob
3   DD-404    5   3.75     A101           2           Bob


In [17]:
# --- Plain text with context manager
poem_path = DATA / "text" / "poem.txt"
with open(poem_path, "r", encoding="utf-8") as f:
    text = f.read()
print(text.splitlines()[:2])  # first 2 lines
with open(poem_path, "r", encoding="utf-8") as f:
    lines = f.readlines()
print("line count:", len(lines))

['If data be the food of thought, import on;', 'Give me excess of files, that, surfeiting,']
line count: 4


In [18]:
# --- HTML tables (requires lxml)
html_path = DATA / "html" / "tables.html"
tables = pd.read_html(str(html_path))  # returns a list of DataFrames
print("Number of tables:", len(tables))
tables[0].head()

ImportError: Missing optional dependency 'lxml'.  Use pip or conda to install lxml.